ATTRACTION - ROBERTA BASE - DATASET ORIGINAL - OPINION

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# tf.config.set_logical_device_configuration(
# gpus[0],
# [tf.config.LogicalDeviceConfiguration(memory_limit=9216  )])

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [3]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset("javilonso/mex_data", use_auth_token=True)

Using custom data configuration javilonso--mex_data-0c030e0c8e30da9e
Reusing dataset parquet (/home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data-0c030e0c8e30da9e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
raw_dataset

DatasetDict({
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 5987
    })
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 23944
    })
})

In [5]:
def tokenize(example):
    tokenized_example = tokenizer(example["Opinion"], truncation=True)
    tokenized_example["label"] = example["Attraction"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [6]:
tokenized_dataset["train"][0]

{'input_ids': [0,
  454,
  1936,
  369,
  7658,
  644,
  364,
  341,
  2762,
  313,
  533,
  7658,
  11062,
  68,
  859,
  7082,
  342,
  387,
  22319,
  644,
  28172,
  66,
  623,
  344,
  1416,
  1233,
  8883,
  11062,
  68,
  4742,
  2762,
  5209,
  341,
  5089,
  344,
  17984,
  467,
  1713,
  342,
  341,
  2090,
  27152,
  313,
  1488,
  55,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': 0}

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [8]:
batch_size = 8

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=batch_size,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=batch_size,
)

In [9]:
model.config.num_labels

3

In [10]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3060, compute capability 8.6


In [11]:
from transformers import create_optimizer

num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback
callback = PushToHubCallback(output_dir="Mex_Rbta_Opinion_Attraction", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs
)

Cloning https://huggingface.co/javilonso/Mex_Rbta_Opinion_Attraction into local empty directory.


Epoch 1/3
2993/2993 [==============================] - 1140s 374ms/step - loss: 0.0863 - val_loss: 0.0476
Epoch 2/3
2993/2993 [==============================] - ETA: 0s - loss: 0.0230

------------------- SKLEARN METRICS -------------------

In [14]:
import numpy as np
from datasets import load_metric
y_test_aux = []
y_pred_aux = []
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    y_test_aux.append(labels)
    y_pred_aux.append(predictions)

# Flatten arrays
y_pred = []
for arr in y_pred_aux:
    for elem in arr:
        y_pred.append(elem)
y_pred = np.array(y_pred)
        
y_test = []
for arr in y_test_aux:
    for elem in arr:
        y_test.append(elem)
y_test = np.array(y_test)

# TEST Predictions
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3265
           1       0.98      0.98      0.98      1667
           2       0.99      0.99      0.99      1055

    accuracy                           0.99      5987
   macro avg       0.99      0.99      0.99      5987
weighted avg       0.99      0.99      0.99      5987



In [ ]:
# import numpy as np
# from datasets import load_metric
 
# load_accuracy = load_metric("accuracy")
# load_f1 = load_metric("f1")
# for batch in tf_eval_dataset:
#     logits = model.predict(batch)["logits"]
#     labels = batch["labels"]
#     predictions = np.argmax(logits, axis=-1)
#     load_accuracy.add_batch(predictions=predictions, references=labels)
#     load_f1.add_batch(predictions=predictions, references=labels)


# accuracy = load_accuracy.compute()["accuracy"]
# f1 = load_f1.compute(average="None")["f1"]

In [ ]:
# print(f'Accuracy: {accuracy:.2f}')
# print(f'F1: {f1:.2f}')


In [ ]:
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# model_name = "classificationEsp1_Attraction"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:
# model.push_to_hub("javilonso/classificationEsp1_Attraction")
# tokenizer.push_to_hub("javilonso/classificationEsp1_Attraction")

In [ ]:
# # Tests ------

# check_point = "javilonso/classificationEsp1_Attraction"

# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# model = TFAutoModelForSequenceClassification.from_pretrained(check_point)
# tokenizer = AutoTokenizer.from_pretrained(check_point)


# from datasets import load_dataset, DatasetDict

# raw_dataset = DatasetDict.load_from_disk(".././data")

# def tokenize(example):
#     tokenized_example = tokenizer(example["Opinion"], truncation=True)
#     tokenized_example["label"] = example["Attraction"]
#     return tokenized_example


# tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)


# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")


# tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
#     columns=["attenti on_mask", "input_ids", "label"],
#     collate_fn=data_collator,
#     shuffle=True,
#     batch_size=16,
# )

# tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "label"],
#     collate_fn=data_collator,
#     shuffle=False,
#     batch_size=16,
# )

# import numpy as np
# from datasets import load_metric
 
# load_accuracy = load_metric("accuracy")
# load_f1 = load_metric("f1")
# for batch in tf_eval_dataset:
#     logits = model.predict(batch)["logits"]
#     labels = batch["labels"]
#     predictions = np.argmax(logits, axis=-1)
#     load_accuracy.add_batch(predictions=predictions, references=labels)
#     load_f1.add_batch(predictions=predictions, references=labels)


# accuracy = load_accuracy.compute()["accuracy"]
# f1 = load_f1.compute(average=None)["f1"]

In [ ]:
# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1: {f1}')
